### Flower Constellations in Sedaro

The code here follows the derivations described in the two references below.

1. Wilkins, M., "The Flower Constellations - Theory, Design Process, and Applications", Doctoral Dissertation, Texas A&M University, 2004.

2. Ruggieri, M., et al. "The Flower Constellation Set and its Possible Applications", ACT Final Report, 2006.

3. Mortari, D., "Flower Constellations as Rigid Objects in Space", First Workshop on Innovative System Concepts, 2006.

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
import json
from sedaro import SedaroApiClient
from math import sqrt, sin, cos, radians, pi, atan, degrees
from scipy.optimize import minimize_scalar

# Constants
Re = 6378.1363        # Earth radius
J2 = 1.0826269e-3     # Earth J2
MU = 398600.4418      # Earth gravitational coefficient
we = 7.2921158553e-5  # Earth angular rate

#### Constellation Configuration

Flower constellations are defined by eight parameters

$$
[N_p, N_d, N_s, F_n, F_d, w, i, h_p]
$$

The code below includes several pre-defined options drawn from references [1] and [2].

In [ ]:
# Configuration
# Np - Nd - Ns - Fn - Fd

Np = 38     # Number of petals (integer)
Nd = 23     # Number of days to repeat (integer)
Ns = 77    # Number of satellites

Fn = 3     # Phasing parameter (integer)
Fd = 5    # Number of distinct orbits (integer)

w = 270    # Argument of perigee (degrees)
i = 0      # Inclination (degrees)
hp = 1300  # Height of perigee (km)


# Pre-defined configurations
# Np, Nd, Ns, Fn, Fd, w, i, hp = 3, 1, 5, 3, 5, 270, 45, 1291.271484      # Basic flower
# Np, Nd, Ns, Fn, Fd, w, i, hp = 8, 1, 90, 1, 90, 270, 165, 3000          # Circles
# Np, Nd, Ns, Fn, Fd, w, i, hp = 31, 18, 80, 18, 80, 270, 63.4, 600       # Helix
# Np, Nd, Ns, Fn, Fd, w, i, hp = 37, 18, 19, 6, 19, 270, 63.4, 19702      # Figure Eight
# Np, Nd, Ns, Fn, Fd, w, i, hp = 37, 18, 19*2, 6, 19*2, 270, 63.4, 19702  # Double Figure Eight
# Np, Nd, Ns, Fn, Fd, w, i, hp = 38, 23, 77, 23, 77, 270, 0, 1300         # Star
# Np, Nd, Ns, Fn, Fd, w, i, hp = 38, 23, 146, 23, 146, 270, 0, 1300       # 8-Pointed Star


# Misc. options
raan_offset = 0          # Right ascension of the initial satellite (degrees)
mean_anomaly_offset = 0  # Mean anomaly of the initial satellite (degrees)
delete_existing = False   # Delete all existing agents in the scenario

In [ ]:
# Scenario Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
# AGENT_TEMPLATE_BRANCH_ID = config['FLOWERS']['AGENT_TEMPLATE_BRANCH_ID']    # ID of a vehicle template branch\n",
AGENT_TEMPLATE_BRANCH_ID = None                                             # Toggle to use a templated agent\n",
SCENARIO_BRANCH_ID = config['FLOWERS']['SCENARIO_BRANCH_ID']                # ID of a new scenario template branch\n",
HOST = config['HOST']                                                       # Sedaro instance URL"

#### Constellation Derivation

Derives the semimajor axis and orbit eccentricity necessary to satisfy the configuration above, then calculates the true anomaly and right ascension of the ascending node for each orbit.

In [ ]:
# Convert to radians for calculations
w = radians(w)
i = radians(i)

# Solve for compatible semimajor axis
def fcn(a):
    e = 1 - (Re + hp) / a
    p = a * (1 - e * e)
    n = sqrt(MU / a**3)
    zeta = 3 * Re * Re * J2 / (4 * p * p)
    tau = Nd / Np
    Az = zeta * (4 + 2 * sqrt(1 - e * e) - (5 + 3 * sqrt(1 - e * e) * sin(i)**2))
    return ((we * (1 - Az)) / (tau - 2 * zeta * cos(i)) - n)**2

result = minimize_scalar(fcn, bounds=(Re + hp, 2.5e6))
if not result.success:
    raise ValueError("Could not solve for semimajor axis!")
else:
    a = result.x
    e = 1 - (Re + hp) / a

    print(f"Parameters found in {result.nit} iterations!")
    print(f"    Semimajor Axis: {a:.3f}km")
    print(f"    Eccentricity: {e:.6f}")

In [ ]:
def mean_anomaly_to_true(M, e):
    E = minimize_scalar(lambda E: (E - e * sin(E) - M)**2, bounds=(0, 2 * pi)).x
    beta = e / (1 + sqrt(1 - e * e))
    return E + 2 * atan(beta * sin(E) / (1 - beta * cos(E)))

mean_anomalies = [mean_anomaly_offset]
true_anomalies = [mean_anomaly_to_true(0, e)]
right_ascensions = [raan_offset]

p = a * (1 - e * e)
zeta = 3 * Re * Re * J2 / (4 * p * p)
tau = Nd / Np
Az = zeta * (4 + 2 * sqrt(1 - e * e) - (5 + 3 * sqrt(1 - e * e) * sin(i)**2))

for _ in range(Ns - 1):
    raan = right_ascensions[-1] - 2 * pi * Nd / Fd
    mean_anomaly = mean_anomalies[-1] + 2 * pi * Np / Fd

    right_ascensions.append(raan % (2 * pi))
    mean_anomalies.append(mean_anomaly % (2 * pi))
    true_anomalies.append(mean_anomaly_to_true(mean_anomalies[-1], e) % (2 * pi))


#### Agent Creation

This code block will create the necessary agents in the target scenario. If `AGENT_BRANCH_TEMPLATE_ID` is defined, the agents will be created with that template. Otherwise, each agent will be a peripheral agent. If `delete_existing` is `True`, the existing agents in the scenario will be wiped out. If `False`, the new agents will be created alongside the existing entries.

In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)

scenario_branch = sedaro.scenario(SCENARIO_BRANCH_ID)

if delete_existing:
    agent_ids = [entry.id for entry in scenario_branch.Agent.get_all()]
    orbit_ids = [entry.id for entry in scenario_branch.Orbit.get_all()]
    if len(agent_ids) + len(orbit_ids) > 0:
        scenario_branch.crud(delete=(agent_ids + orbit_ids))
    agent_id_offset = 0
else:
    agent_id_offset = len(scenario_branch.Agent.get_all())

orbits, agents = [], []
for idx, (raan, true_anomaly) in enumerate(zip(right_ascensions, true_anomalies)):
    orbit = dict(
        type='Orbit',
        initialStateDefType='ORBITAL_ELEMENTS',
        initialStateDefParams={
            'a': a,
            'e': e,
            'inc': degrees(i),
            'om': degrees(w),
            'nu': degrees(true_anomaly),
            'raan': degrees(raan),
        }
    )
    agent = dict(
        type='Agent',
        name=f'Sat-{idx + agent_id_offset}',
        peripheral=True,
        differentiatingState={
            'targetType': 'SpaceTarget'
        },
    )

    if AGENT_TEMPLATE_BRANCH_ID is not None:
        agent['peripheral'] = False
        agent['templateRef'] = AGENT_TEMPLATE_BRANCH_ID
    orbits.append(orbit)
    agents.append(agent)

result = scenario_branch.crud(blocks=orbits)
orbit_ids = result['crud']['blocks']
for orbit_id, agent in zip(orbit_ids, agents):
    agent['orbit'] = orbit_id
result = scenario_branch.crud(blocks=agents)
agent_ids = result['crud']['blocks']

In [ ]:
# Display orbits
col_width = 16
width = (col_width + 1) * 6 + 1
headers = ['a', 'e', 'i', 'w', 'nu', 'raan']

print(f'{len(mean_anomalies)} Created Orbits'.center(width))
print('-' * width)
print('|' + '|'.join(entry.center(col_width) for entry in headers) + '|')
print('-' * width)
for raan, nu in zip(right_ascensions, true_anomalies):
    print(
        f'| {a:>14.6f} | {e:>14.12f} | {degrees(i):>14.10f} | {degrees(w):>14.10f} '
        f'| {degrees(nu):>14.10f} | {degrees(raan):14.10f} |'
    )

#### Cleanup

In [ ]:
# Delete created Agents and Orbits
scenario_branch.crud(delete=agent_ids + orbit_ids)